<a href="https://colab.research.google.com/github/mz20082/CE888-Assignment1/blob/main/Emoji_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the files necessary for the analysis

In [1]:
from google.colab import files 
uploaded = files.upload()

Saving mapping.txt to mapping.txt
Saving test_labels.txt to test_labels.txt
Saving test_text.txt to test_text.txt
Saving train_labels.txt to train_labels.txt
Saving train_text.txt to train_text.txt
Saving val_labels.txt to val_labels.txt
Saving val_text.txt to val_text.txt


In [ ]:
#installing NLTK for preprocessing

In [3]:
pip install nltk

In [ ]:
#importing pandas,numpy, random,nltk,re and sklearn

In [4]:
import pandas as pd
import numpy as np
import random
import nltk
import re
import sklearn
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#installing emoji's

In [8]:
!pip install emoji


     |████████████████████████████████| 133kB 4.4MB/s 


In [ ]:
#importing emoji's as emoji

In [9]:
import emoji as emoji

In [ ]:
#creating the emoji dictionary

In [10]:

emoji_dictionary = {"0": "\u2764\uFE0F",    
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":chestnut:"
                   }

In [11]:

emoji.emojize(":chestnut:")

'🌰'

In [12]:
for e in emoji_dictionary.values():           # print all emoji in the list
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴
💯
🔥
🌰


In [14]:
#reading the training and test data set using read_csv
train = pd.read_csv('train_text.txt',error_bad_lines=False)
test = pd.read_csv('test_text.txt',error_bad_lines=False)

b'Skipping line 293: expected 3 fields, saw 4\nSkipping line 306: expected 3 fields, saw 5\nSkipping line 427: expected 3 fields, saw 4\nSkipping line 611: expected 3 fields, saw 5\nSkipping line 969: expected 3 fields, saw 4\nSkipping line 1042: expected 3 fields, saw 4\nSkipping line 1361: expected 3 fields, saw 4\nSkipping line 1383: expected 3 fields, saw 9\nSkipping line 1430: expected 3 fields, saw 4\nSkipping line 1469: expected 3 fields, saw 4\nSkipping line 1998: expected 3 fields, saw 4\nSkipping line 2527: expected 3 fields, saw 4\nSkipping line 2662: expected 3 fields, saw 4\nSkipping line 2721: expected 3 fields, saw 4\nSkipping line 2845: expected 3 fields, saw 4\nSkipping line 3467: expected 3 fields, saw 5\nSkipping line 3563: expected 3 fields, saw 5\nSkipping line 3594: expected 3 fields, saw 4\nSkipping line 3617: expected 3 fields, saw 4\nSkipping line 3966: expected 3 fields, saw 4\nSkipping line 4291: expected 3 fields, saw 4\nSkipping line 4891: expected 3 fields

In [19]:
#renaming the column 
train.columns = ['tweet','tweet1']
train

,tweet,tweet1
Time for some BBQ and whiskey libations. Chomp,belch,chomp! (@ Lucille's Smokehouse Bar-B-Que)
Love love love all these people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ San…,NaN,NaN
"️ ️ ️ ️ @ Toys""R""Us",NaN,NaN
Man these are the funniest kids ever!! That face! #HappyBirthdayBubb @ FLIPnOUT Xtreme,NaN,NaN
#sandiego @ San Diego,California,NaN
...,...,...
Here to celebrate the Nunez wedding! Love my best friends ️ @user,NaN,NaN
1 night in Paris.... Wait... @ Paris Las Vegas Hotel &amp; Casino,NaN,NaN
Be safe this weekend everyone. #happylaborday @ Beer NV,NaN,NaN
Pizza (@ Five50 - @user in Las Vegas,NV),NaN


In [36]:
#renaming the test column
test.columns = ['tweet','tweet1']

In [21]:
#checking the null values
train.isnull().sum()

tweet     32446
tweet1    41373
dtype: int64

In [23]:
#reading the train and test labels
tr = pd.read_csv('train_labels.txt',error_bad_lines=False)
ts = pd.read_csv('test_labels.txt',error_bad_lines=False)  

In [27]:
#renaming the train and test labels
tr.columns = ['label']
ts.columns = ['label']

In [32]:
#using the concat function to concat the train and train label
train = pd.concat([tr,train])
train

,label,tweet,tweet1
0,19.0,NaN,NaN
1,0.0,NaN,NaN
2,0.0,NaN,NaN
3,2.0,NaN,NaN
4,11.0,NaN,NaN
...,...,...,...
Here to celebrate the Nunez wedding! Love my best friends ️ @user,NaN,NaN,NaN
1 night in Paris.... Wait... @ Paris Las Vegas Hotel &amp; Casino,NaN,NaN,NaN
Be safe this weekend everyone. #happylaborday @ Beer NV,NaN,NaN,NaN
Pizza (@ Five50 - @user in Las Vegas,NaN,NV),NaN


In [37]:
# using the concat function to concat the test and test label
test = pd.concat([test,ts])
test

,tweet,tweet1,label
0,The calm before...... | w/ sofarsounds @user |...,NaN,NaN
1,CHRISTMAS DEALS BUY ANY 3 SMALL POMADES 1.5 OR...,NaN,NaN
2,the #sisterstunt was mad real last night #MiaS...,NaN,NaN
3,Let the sun shine through ️ 5x5 Feet #oilpaint...,NaN,NaN
4,Still bitch im trill never been no fiend :@vib...,NaN,NaN
...,...,...,...
49994,NaN,NaN,5.0
49995,NaN,NaN,19.0
49996,NaN,NaN,6.0
49997,NaN,NaN,0.0


In [43]:
# Converting Sentences into Vector (Embedding layer output
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output